In [ ]:
import ROOT
from math import log, sqrt

# silence most of the roofit message (bug in ROOT saturate jupyter stream)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.NumIntegration)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.Fitting)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.Minimization)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.InputArguments)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.Eval)
ROOT.RooMsgService.instance().setGlobalKillBelow(ROOT.RooFit.ERROR)

ROOT.RooStats.AsymptoticCalculator.SetPrintLevel(-1)

ROOT.enableJSVis()

In [ ]:
f = ROOT.TFile.Open('../data/workspace.root')
ws = f.Get("ws")
ws.Print()

## Compute intervals on POI and plot profile likelihood ratio

Here we are doing a measurement, in particular we are plotting the confidence level for the cross section using the ProfileLikelihoodCalculator. We are using real data. The study is repeated with / without systematics.

In [ ]:
# when doing interference you need only model config and data
model_config = ws.obj('model_config')    # can also use the s+b model config
data = ws.data('data_binned')            # real data

canvas = ROOT.TCanvas()
poi = model_config.GetParametersOfInterest().first()  # the cross section
# do an initial fit
model_config.GetPdf().fitTo(data, ROOT.RooFit.PrintLevel(-1))

# create a profile likelihood calculator (it just compute the profile
# likelihood, can perform a scan and find a confidence interval
pl = ROOT.RooStats.ProfileLikelihoodCalculator(data, model_config)
pl.SetConfidenceLevel(0.683)  # usual 1 sigma gaussian
interval = pl.GetInterval()   # the real work
lowerLimit = interval.LowerLimit(poi)
upperLimit = interval.UpperLimit(poi)
print "[lower, upper] range: [%.3f, %.3f]" % (lowerLimit, upperLimit)
plot = ROOT.RooStats.LikelihoodIntervalPlot(interval)
plot.Draw("tf1")

model_config.GetPdf().fitTo(data, ROOT.RooFit.PrintLevel(-1))
# set the pull to be constant, in this way there are no systematic in the model
# the error we get is the error as there were no systematic at all
# we assume it to be the statistical error
ROOT.RooStats.SetAllConstant(ws.set('pulls'))
pl_nosys = ROOT.RooStats.ProfileLikelihoodCalculator(data, model_config)
pl_nosys.SetConfidenceLevel(0.683)  # usual 1 sigma gaussian
interval_nosys = pl_nosys.GetInterval()
lowerLimit_nosys = interval_nosys.LowerLimit(poi)
upperLimit_nosys = interval_nosys.UpperLimit(poi)
print "[lower, upper] range: [%.3f, %.3f]" % (lowerLimit_nosys, upperLimit_nosys)
interval.GetBestFitParameters().Print("V")
plot_nosys = ROOT.RooStats.LikelihoodIntervalPlot(interval_nosys)
plot_nosys.Draw("tf1 same")
ROOT.RooStats.SetAllConstant(ws.set('pulls'), False)
canvas.Draw()

ProfileLikelihoodCalculator is not very used since it is not very flexible: difficult to tweak the profile likelihood scan plot, not possible to parallelize the computation of the likelihood for each point of the scan... Usually people do a manual scan, usually splitting the computation on different machines.

Write a loop to do the scan using the profile likelihood object

In [ ]:
model_config = ws.obj('model_config')
data = ws.data('data_binned')

nll = model_config.GetPdf().createNLL(data)
poi = model_config.GetParametersOfInterest().first()
pnll = nll.createProfile(ROOT.RooArgSet(poi))

# compute the value of the profile likelihood ratio for xsection == 0
poi.setVal(0)
print pnll.getVal()  # this is the value of the profile likelihood for a particular value of the poi

# plot it
canvas = ROOT.TCanvas()
# define a frame with the xsection on the xaxis
frame = poi.frame()
# plot the profile likelihood ratio on the frame
pnll.plotOn(frame)
frame.Draw()
canvas.Draw()

# repeat with no systematics (fixing NP to best values, to get the best value do a fit first)

***EXERCIZE***: what is the significance of the excess from the previous plot?

***EXERCIZE***: scan the mass $m_H$

In [ ]:
# FILL HERE